1. Ingesting the JSON file

In [0]:
account_key = dbutils.secrets.get(scope="formala1-scope", key="account-key")
spark.conf.set("fs.azure.account.key.formala1datalake.dfs.core.windows.net", account_key)

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date","")
v_file_date = dbutils.widgets.get("p_file_date")
print(v_file_date)

In [0]:
constructor_schema = "constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"
constructor_df = spark.read.json(f"abfss://raw@formala1datalake.dfs.core.windows.net/{v_file_date}/constructors.json",schema=constructor_schema)

In [0]:
display(constructor_df)

In [0]:
constructor_df.printSchema()

In [0]:
constructors_df = constructor_df.drop("url")

In [0]:
from pyspark.sql.functions import current_timestamp, lit
constructor_final_df = constructors_df.withColumnRenamed("constructorId", "constructor_id").withColumnRenamed("constructorRef", "constructor_ref").withColumn("ingestion_date",current_timestamp()).withColumn("file_date", lit(v_file_date))

In [0]:
constructor_final_df.write.mode("append").format("delta").saveAsTable("f1_processed.constructors")

In [0]:
%sql
select * from f1_processed.constructors